In [42]:
from gym import Env
from gym.spaces import Discrete, Box, Tuple
from random import random
import numpy as np
from create_dataset import CreateDataset

In [48]:
# Building a custom environment
class FootballEnv(Env):
    def __init__(self, observations):
        self.dataset = CreateDataset()
        self.counter = 0
        self.observations = observations

        self.sequence_length = 5
        self.steps_left = self.sequence_length

        # The amount of actions that a player can take
        self.action_size = len(self.dataset.good_events)

        # The currect action being taken
        self.state = None

        # The possible action space ( A discrete action )
        self.action_space = Discrete(self.action_size)

        # The observation ( The last 5 actions )
        # [0] - The action being taken
        # [1] - The row
        # [2] - The column
        self.observation_space = np.array(
            Tuple((
                Discrete(self.action_size), 
                Discrete(self.dataset.xT_Map.shape[0]),
                Discrete(self.dataset.xT_Map.shape[1]),

                Discrete(self.action_size), 
                Discrete(self.dataset.xT_Map.shape[0]),
                Discrete(self.dataset.xT_Map.shape[1]),

                Discrete(self.action_size), 
                Discrete(self.dataset.xT_Map.shape[0]),
                Discrete(self.dataset.xT_Map.shape[1]),

                Discrete(self.action_size), 
                Discrete(self.dataset.xT_Map.shape[0]),
                Discrete(self.dataset.xT_Map.shape[1]),
                
                Discrete(self.action_size), 
                Discrete(self.dataset.xT_Map.shape[0]),
                Discrete(self.dataset.xT_Map.shape[1]),
            ))
        )
        # self.observation_space = np.array(
        #     Tuple((
        #         Tuple((Discrete(self.action_size), Discrete(self.dataset.xT_Map.shape[0]), Discrete(self.dataset.xT_Map.shape[1]))),
        #         Tuple((Discrete(self.action_size), Discrete(self.dataset.xT_Map.shape[0]), Discrete(self.dataset.xT_Map.shape[1]))),
        #         Tuple((Discrete(self.action_size), Discrete(self.dataset.xT_Map.shape[0]), Discrete(self.dataset.xT_Map.shape[1]))),
        #         Tuple((Discrete(self.action_size), Discrete(self.dataset.xT_Map.shape[0]), Discrete(self.dataset.xT_Map.shape[1]))),
        #         Tuple((Discrete(self.action_size), Discrete(self.dataset.xT_Map.shape[0]), Discrete(self.dataset.xT_Map.shape[1])))
        #     ))
        # )

    def step(self, action):
        self.steps_left -= 1
        reward = random()
        self.counter += 1

        self.state = self.observations[self.counter][3:].tolist()
        self.state.append(action)
        self.state.append(0)
        self.state.append(0)

        self.state = np.zeros((len(self.observation_space)), dtype=np.float32)
        self.state[action] = 1
        info = {}
        return self.state, reward, self.steps_left == 0, info

    def render(self):
        pass

    def reset(self):
        self.sequence_length = 5
        self.steps_left = self.sequence_length
        self.state = np.zeros(len(self.observation_space), dtype=np.float32)
        return self.state

In [51]:
def create_dataset():

    dataset_maker = CreateDataset()
    dataset_maker.loadFile('data.json')

    observations, actions, rewards = dataset_maker.createEpisodeDataset()
    terminals = np.array([ 0 if (i+1) % dataset_maker.lim == 0 else 1 for i in range(len(actions)) ])
    return observations